<h1>Deep Learning</h1>

(Small advertising: don't activate the environment from base environment, think to first deactivate)

In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mlxtend.data import loadlocal_mnist
from keras.utils import np_utils
from keras.optimizers import SGD, Adam, RMSprop
from keras.losses import categorical_crossentropy
from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.models import Sequential
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras.datasets import cifar10

from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras import backend as K
K.set_image_dim_ordering(dim_ordering="th") #we want to use Theano backend image dim order
print(K.backend()) #we use Theano as backend
#we can see that Theano is used in /media/anthony/USOPP/conda/IA/etc/conda/activate.d/keras_activate.sh

import tensorboard
%matplotlib inline
np.random.seed(seed=777)

theano


In [2]:
MNIST_PATH = '/media/anthony/USOPP/mnist_data'

<h2>1) Multilayer Perceptron</h2>
MNIST dataset at <a href='http://yann.lecun.com/exdb/mnist/', target='_blank'>http://yann.lecun.com/exdb/mnist/</a>

In [25]:
X_train, Y_train = loadlocal_mnist(images_path=f'{MNIST_PATH}/train-images-idx3-ubyte',
                                   labels_path=f'{MNIST_PATH}/train-labels-idx1-ubyte')

X_test, Y_test = loadlocal_mnist(images_path=f'{MNIST_PATH}/t10k-images-idx3-ubyte',
                                 labels_path=f'{MNIST_PATH}/t10k-labels-idx1-ubyte')

print(X_train.shape) #Originally (60000, 28, 28) dimension (28px x 28px images)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(60000, 784)
(60000,)
(10000, 784)
(10000,)


<h4>Processing</h4>

In [26]:
X_train = X_train.astype('float32') #for rescaling part
X_test = X_test.astype('float32')
Y_train = np_utils.to_categorical(y=Y_train, num_classes=10)
Y_test = np_utils.to_categorical(y=Y_test, num_classes=10)

#rescaling
X_train /= 255
X_test /= 255

<h4>Modelling</h4>

In [27]:
NB_EPOCHS = 50
NB_CLASSES = 10
OPTIMIZER = SGD() #algorithm for weight optimization during training
BATCH_SIZE = 128
N_HIDDEN = 100
VALIDATION_SPLIT = 0.2 #for fine-tuning before the assesment with the test-set
LOSS_FCT = 'categorical_crossentropy'
DROPOUT = 0.3

In [28]:
model = Sequential() #input layer already implemented
model.add(Dense(units=N_HIDDEN, activation="relu", input_dim=784)) #first hidden layer
model.add(Dropout(rate=DROPOUT))
model.add(Dense(units=N_HIDDEN, activation="relu")) #second hidden layer
model.add(Dense(units=NB_CLASSES, activation="softmax")) #output layer
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               78500     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1010      
Total params: 89,610
Trainable params: 89,610
Non-trainable params: 0
_________________________________________________________________


In [29]:
#compiling the model
model.compile(optimizer=OPTIMIZER, loss=LOSS_FCT, metrics=['accuracy'])

#training the model
model.fit(x=X_train, y=Y_train, batch_size=BATCH_SIZE,
          epochs=NB_EPOCHS, validation_split=VALIDATION_SPLIT)

Train on 48000 samples, validate on 12000 samples
Epoch 1/50
48000/48000 [==============================] - 2s 45us/step - loss: 1.7056 - acc: 0.4920 - val_loss: 0.8977 - val_acc: 0.8083
Epoch 2/50
48000/48000 [==============================] - 2s 37us/step - loss: 0.8226 - acc: 0.7624 - val_loss: 0.5144 - val_acc: 0.8701
Epoch 3/50
13952/48000 [=======>......................] - ETA: 1s - loss: 0.6542 - acc: 0.8064

KeyboardInterrupt: 

In [ ]:
test_score = model.evaluate(x=X_test, y=Y_test)
print(f'Loss function minima for testing set : {test_score[0]:.3f}')
print(f'Accuracy maxima for testing set: {test_score[1]:.3f}')

In [ ]:
train_score = model.evaluate(x=X_train, y=Y_train)
print(f'Loss function minima for training set : {train_score[0]:.3f}')
print(f'Accuracy maxima for training set: {train_score[1]:.3f}')